In [1]:
import win32com.client as win32
import openpyxl
import pandas as pd
import ctypes
import win32com
import time
import shutil
import warnings
import os
from pathlib import Path
import numpy as np
import datetime
import time
from openpyxl import Workbook
from openpyxl.styles import Alignment
from openpyxl.styles import Font
from openpyxl.styles import PatternFill

print("Создание отчета общего по просрочке+прогноз на неделю")
dd = datetime.date.today().strftime("%d.%m.%Y")
df=pd.read_excel(f'file_cov/ВВБ_AntiCOV_v2_{dd}.xlsx', sheet_name='Лист1', 
               usecols=["ВСП/КИЦ", "Блок 2", "ФИО", "Должность", "Принадлежность", "Прогноз ревакцинации на неделю", "Просрочена ревакцинация", "Подразделение кратко", "Кол-во дней до ревакцинации (с уч.мед-да)"])

#выгрузка данных в шаблон wsa из Коллективного иммунитета
df["ФИО"]=np.where((df["Просрочена ревакцинация"]==1)|(df["Прогноз ревакцинации на неделю"]==1), df["ФИО"], np.nan)
df=df[df["ФИО"].notnull()]
    
del df["Просрочена ревакцинация"]
del df["Прогноз ревакцинации на неделю"]
#new
df.insert(0, "Номер", "")
df["Номер"]=range(1, len(df)+1)
df.insert(1, "Регион", df["Принадлежность"])
df.insert(2, "Блок", df["Блок 2"])

df.loc[df["Кол-во дней до ревакцинации (с уч.мед-да)"]=="-", "Кол-во дней до ревакцинации (с уч.мед-да)" ]=(-365)
df["Кол-во дней до ревакцинации (с уч.мед-да)"].fillna((-365), inplace=True)
del df["Блок 2"]
#del df["ВСП/КИЦ"]
del df["Принадлежность"]
#new2

df.to_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx', index=False)

print('Для АТБ общий готов')
#готово1

df=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')

#выгрузка данных в шаблон wsa из Коллективного иммунитета
df["ФИО"]=np.where((df["Регион"]=="АТБ")&(df["Блок"]=="ПРПА"), df["ФИО"], np.nan)
df=df[df["ФИО"].notnull()]

df=df.sort_values(by=["Подразделение кратко"])
df["Номер"]=range(1, len(df)+1)
del df["ВСП/КИЦ"]

df.to_excel(f'pr_rev_BlokATB/ПРПА_ревакцинация_АТБ_{dd}.xlsx', index=False)
print('Для АТБ ПРПА готов')

#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_BlokATB/ПРПА_ревакцинация_АТБ_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 8
ws.column_dimensions['C'].width = 8
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 25
ws.column_dimensions['F'].width = 25
ws.column_dimensions['G'].width = 20
rows = ws.max_row 
for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=2).value=='АТБ':
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill

wb.save(f'pr_rev_BlokATB/ПРПА_ревакцинация_АТБ_{dd}.xlsx')

#выгрузка данных
df2=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')
df2["ФИО"]=np.where((df2["Регион"]=="АТБ")&((df2["Блок"]=="РБиСП (ПЦП)")|(df2["Блок"]=="РБиСП (ЦА)")|(df2["Блок"]=="РБиСП")), df2["ФИО"], np.nan)
df2=df2[df2["ФИО"].notnull()]

df2=df2.sort_values(by=["Блок"])
df2=df2.sort_values(by=["Подразделение кратко"])
df2["Номер"]=range(1, len(df2)+1)    
del df2["ВСП/КИЦ"]

df2.to_excel(f'pr_rev_BlokATB/РБиСП_ревакцинация_АТБ_{dd}.xlsx', index=False)

#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_BlokATB/РБиСП_ревакцинация_АТБ_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
rows = ws.max_row 
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 8
ws.column_dimensions['C'].width = 8
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 38
ws.column_dimensions['F'].width = 35
ws.column_dimensions['G'].width = 20

for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=2).value=='АТБ':
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill
            
wb.save(f'pr_rev_BlokATB/РБиСП_ревакцинация_АТБ_{dd}.xlsx')

print('Для АТБ РБиСП готов')

df3=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')
df3["ФИО"]=np.where((df3["Регион"]=="АТБ")&((df3["Блок"]=="КИБ (ПЦП)")|(df3["Блок"]=="КИБ (ЦА)")|(df3["Блок"]=="КИБ")), df3["ФИО"], np.nan)
df3=df3[df3["ФИО"].notnull()]

df3=df3.sort_values(by=["Блок"])
df3=df3.sort_values(by=["Подразделение кратко"])
df3["Номер"]=range(1, len(df3)+1)
del df3["ВСП/КИЦ"]

df3.to_excel(f'pr_rev_BlokATB/КИБ_ревакцинация_АТБ_{dd}.xlsx', index=False)

#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_BlokATB/КИБ_ревакцинация_АТБ_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 8
ws.column_dimensions['C'].width = 8
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 38
ws.column_dimensions['F'].width = 35
ws.column_dimensions['G'].width = 20
rows = ws.max_row 
for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=2).value=='АТБ':
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill
            
wb.save(f'pr_rev_BlokATB/КИБ_ревакцинация_АТБ_{dd}.xlsx')

print('Для АТБ КИБ готов')

df4=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')
df4["ФИО"]=np.where((df4["Регион"]=="АТБ")&((df4["Блок"]=="Сервисы (ПЦП)")|(df4["Блок"]=="Сервисы (ЦА)")|(df4["Блок"]=="Сервисы")|(df4["Блок"]=="Технологии (ЦА)")), df4["ФИО"], np.nan)
df4=df4[df4["ФИО"].notnull()]
    
df4=df4.sort_values(by=["Блок"])
df4=df4.sort_values(by=["Подразделение кратко"])
df4["Номер"]=range(1, len(df4)+1)
del df4["ВСП/КИЦ"]

df4.to_excel(f'pr_rev_BlokATB/Сервисы_ревакцинация_АТБ_{dd}.xlsx', index=False)

#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_BlokATB/Сервисы_ревакцинация_АТБ_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 8
ws.column_dimensions['C'].width = 8
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 38
ws.column_dimensions['F'].width = 35
ws.column_dimensions['G'].width = 20
rows = ws.max_row 
for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=2).value=='АТБ':
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill
            
wb.save(f'pr_rev_BlokATB/Сервисы_ревакцинация_АТБ_{dd}.xlsx')

print('Для АТБ Сервисы готов')

df5=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')
df5["ФИО"]=np.where((df5["Регион"]=="АТБ")&((df5["Блок"]=="Блок Председателя")|(df5["Блок"]=="HR (ПЦП)")), df5["ФИО"], np.nan)
df5=df5[df5["ФИО"].notnull()]
    
df5=df5.sort_values(by=["Блок"])
df5=df5.sort_values(by=["Подразделение кратко"])
df5["Номер"]=range(1, len(df5)+1)
del df5["ВСП/КИЦ"]

df5.to_excel(f'pr_rev_BlokATB/Блок Председателя_ревакцинация_АТБ_{dd}.xlsx', index=False)
#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_BlokATB/Блок Председателя_ревакцинация_АТБ_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 8
ws.column_dimensions['C'].width = 16
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 38
ws.column_dimensions['F'].width = 35
ws.column_dimensions['G'].width = 20
rows = ws.max_row 
for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=2).value=='АТБ':
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill
            
wb.save(f'pr_rev_BlokATB/Блок Председателя_ревакцинация_АТБ_{dd}.xlsx')

print('Для АТБ Блок Председателя готов')

#по Принадлежности

df=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')
df["ФИО"]=np.where((df["Регион"]=="АТБ"), df["ФИО"], np.nan)
df=df[df["ФИО"].notnull()]

df=df.sort_values(by=["Блок"])
df=df.sort_values(by=["Подразделение кратко"])    
df["Номер"]=range(1, len(df)+1)
del df["ВСП/КИЦ"]

df.to_excel(f'pr_rev_Terr/Ревакцинация_АТБ_{dd}.xlsx', index=False)
#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_Terr/Ревакцинация_АТБ_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 8
ws.column_dimensions['C'].width = 16
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 38
ws.column_dimensions['F'].width = 35
ws.column_dimensions['G'].width = 20
rows = ws.max_row 
for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=2).value=='АТБ':
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill
            
wb.save(f'pr_rev_Terr/Ревакцинация_АТБ_{dd}.xlsx')

print('Для АТБ готов')

df2=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')
df2["ФИО"]=np.where((df2["Регион"]=="ВО"), df2["ФИО"], np.nan)
df2=df2[df2["ФИО"].notnull()]

df2=df2.sort_values(by=["Блок"])
df2=df2.sort_values(by=["Подразделение кратко"])    
df2["Номер"]=range(1, len(df2)+1)
del df2["ВСП/КИЦ"]

df2.to_excel(f'pr_rev_Terr/Ревакцинация_ВО_{dd}.xlsx', index=False)
#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_Terr/Ревакцинация_АТБ_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 8
ws.column_dimensions['C'].width = 16
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 38
ws.column_dimensions['F'].width = 35
ws.column_dimensions['G'].width = 20
rows = ws.max_row 
for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=2).value=='ВО':
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill
            
wb.save(f'pr_rev_Terr/Ревакцинация_ВО_{dd}.xlsx')

print('Для ВО готов')

df3=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')
df3["ФИО"]=np.where((df3["Регион"]=="КО"), df3["ФИО"], np.nan)
df3=df3[df3["ФИО"].notnull()]

df3=df3.sort_values(by=["Блок"])
df3=df3.sort_values(by=["Подразделение кратко"])    
df3["Номер"]=range(1, len(df3)+1)
del df3["ВСП/КИЦ"]

df3.to_excel(f'pr_rev_Terr/Ревакцинация_КО_{dd}.xlsx', index=False)
#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_Terr/Ревакцинация_КО_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 8
ws.column_dimensions['C'].width = 16
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 38
ws.column_dimensions['F'].width = 35
ws.column_dimensions['G'].width = 20
rows = ws.max_row 
for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=2).value=='КО':
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill
            
wb.save(f'pr_rev_Terr/Ревакцинация_КО_{dd}.xlsx')

print('Для КО готов')

df4=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')
df4["ФИО"]=np.where((df4["Регион"]=="МЭ"), df4["ФИО"], np.nan)
df4=df4[df4["ФИО"].notnull()]

df4=df4.sort_values(by=["Блок"])
df4=df4.sort_values(by=["Подразделение кратко"])
df4["Номер"]=range(1, len(df4)+1)
del df4["ВСП/КИЦ"]

df4.to_excel(f'pr_rev_Terr/Ревакцинация_МЭ_{dd}.xlsx', index=False)
#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_Terr/Ревакцинация_МЭ_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 8
ws.column_dimensions['C'].width = 16
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 38
ws.column_dimensions['F'].width = 35
ws.column_dimensions['G'].width = 20
rows = ws.max_row 
for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=2).value=='МЭ':
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill
            
wb.save(f'pr_rev_Terr/Ревакцинация_МЭ_{dd}.xlsx')

print('Для МЭ готов')

df5=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')
df5["ФИО"]=np.where((df5["Регион"]=="НО"), df5["ФИО"], np.nan)
df5=df5[df5["ФИО"].notnull()]

df5=df5.sort_values(by=["Блок"])
df5=df5.sort_values(by=["Подразделение кратко"])    
df5["Номер"]=range(1, len(df5)+1)
del df5["ВСП/КИЦ"]

df5.to_excel(f'pr_rev_Terr/Ревакцинация_НО_{dd}.xlsx', index=False)

#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_Terr/Ревакцинация_НО_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 8
ws.column_dimensions['C'].width = 16
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 38
ws.column_dimensions['F'].width = 35
ws.column_dimensions['G'].width = 20
rows = ws.max_row 
for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=2).value=='НО':
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill
wb.save(f'pr_rev_Terr/Ревакцинация_НО_{dd}.xlsx')
print('Для НО готов')

df6=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')
df6["ФИО"]=np.where((df6["Регион"]=="ПК"), df6["ФИО"], np.nan)
df6=df6[df6["ФИО"].notnull()]

df6=df6.sort_values(by=["Блок"])
df6=df6.sort_values(by=["Подразделение кратко"])    
df6["Номер"]=range(1, len(df6)+1)
del df6["ВСП/КИЦ"]

df6.to_excel(f'pr_rev_Terr/Ревакцинация_ПК_{dd}.xlsx', index=False)
#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_Terr/Ревакцинация_ПК_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 8
ws.column_dimensions['C'].width = 16
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 38
ws.column_dimensions['F'].width = 35
ws.column_dimensions['G'].width = 20
rows = ws.max_row 
for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=2).value=='ПК':
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill
wb.save(f'pr_rev_Terr/Ревакцинация_ПК_{dd}.xlsx')
print('Для ПК готов')

df7=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')
df7["ФИО"]=np.where((df7["Регион"]=="ПЦП НН (Сахарова и Костина)"), df7["ФИО"], np.nan)
df7=df7[df7["ФИО"].notnull()]

df7=df7.sort_values(by=["Блок"])
df7=df7.sort_values(by=["Подразделение кратко"])
df7["Номер"]=range(1, len(df7)+1)
del df7["ВСП/КИЦ"]

df7.to_excel(f'pr_rev_Terr/Ревакцинация_ПЦП_{dd}.xlsx', index=False)
#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_Terr/Ревакцинация_ПЦП_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 24
ws.column_dimensions['C'].width = 16
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 38
ws.column_dimensions['F'].width = 35
ws.column_dimensions['G'].width = 20
rows = ws.max_row 
for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=2).value=="ПЦП НН (Сахарова и Костина)":
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill
wb.save(f'pr_rev_Terr/Ревакцинация_ПЦП_{dd}.xlsx')
print('Для ПЦП НН готов')

df8=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')
df8["ФИО"]=np.where((df8["Регион"]=="РМ"), df8["ФИО"], np.nan)
df8=df8[df8["ФИО"].notnull()]

df8=df8.sort_values(by=["Блок"])
df8=df8.sort_values(by=["Подразделение кратко"])
df8["Номер"]=range(1, len(df8)+1)
del df8["ВСП/КИЦ"]

df8.to_excel(f'pr_rev_Terr/Ревакцинация_РМ_{dd}.xlsx', index=False)\
#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_Terr/Ревакцинация_РМ_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 8
ws.column_dimensions['C'].width = 16
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 38
ws.column_dimensions['F'].width = 35
ws.column_dimensions['G'].width = 20
rows = ws.max_row 
for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=2).value=='РМ':
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill
wb.save(f'pr_rev_Terr/Ревакцинация_РМ_{dd}.xlsx')

print('Для РМ готов')

df9=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')
df9["ФИО"]=np.where((df9["Регион"]=="РТ"), df9["ФИО"], np.nan)
df9=df9[df9["ФИО"].notnull()]

df9=df9.sort_values(by=["Блок"])
df9=df9.sort_values(by=["Подразделение кратко"])
df9["Номер"]=range(1, len(df9)+1)
del df9["ВСП/КИЦ"]

df9.to_excel(f'pr_rev_Terr/Ревакцинация_РТ_{dd}.xlsx', index=False)
#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_Terr/Ревакцинация_РТ_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 8
ws.column_dimensions['C'].width = 16
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 38
ws.column_dimensions['F'].width = 35
ws.column_dimensions['G'].width = 20
rows = ws.max_row 
for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=2).value=='РТ':
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill
wb.save(f'pr_rev_Terr/Ревакцинация_РТ_{dd}.xlsx')

print('Для РТ готов')

df10=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')
df10["ФИО"]=np.where((df10["Регион"]=="РУ"), df10["ФИО"], np.nan)
df10=df10[df10["ФИО"].notnull()]

df10=df10.sort_values(by=["Блок"])
df10=df10.sort_values(by=["Подразделение кратко"])
df10["Номер"]=range(1, len(df10)+1)
del df10["ВСП/КИЦ"]

df10.to_excel(f'pr_rev_Terr/Ревакцинация_РУ_{dd}.xlsx', index=False)
#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_Terr/Ревакцинация_РУ_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 8
ws.column_dimensions['C'].width = 16
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 38
ws.column_dimensions['F'].width = 35
ws.column_dimensions['G'].width = 20
rows = ws.max_row 
for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=2).value=='РУ':
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill
wb.save(f'pr_rev_Terr/Ревакцинация_РУ_{dd}.xlsx')
print('Для РУ готов')

df11=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')
df11["ФИО"]=np.where((df11["Регион"]=="РЧ"), df11["ФИО"], np.nan)
df11=df11[df11["ФИО"].notnull()]

df11=df11.sort_values(by=["Блок"])
df11=df11.sort_values(by=["Подразделение кратко"])
df11["Номер"]=range(1, len(df11)+1)
del df11["ВСП/КИЦ"]

df11.to_excel(f'pr_rev_Terr/Ревакцинация_РЧ_{dd}.xlsx', index=False)

#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_Terr/Ревакцинация_РЧ_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 8
ws.column_dimensions['C'].width = 16
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 38
ws.column_dimensions['F'].width = 35
ws.column_dimensions['G'].width = 20
rows = ws.max_row 
for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=2).value=='РЧ':
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill
wb.save(f'pr_rev_Terr/Ревакцинация_РЧ_{dd}.xlsx')
print('Для РЧ готов')

df=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')

#выгрузка данных в шаблон wsa из Коллективного иммунитета
df["ФИО"]=np.where((df["ВСП/КИЦ"]=="КИЦ"), df["ФИО"], np.nan)
df=df[df["ФИО"].notnull()]

df.insert(1, "КИЦ", df["ВСП/КИЦ"])
df=df.sort_values(by=["Регион"])
df=df.sort_values(by=["Подразделение кратко"])
df["Номер"]=range(1, len(df)+1)    
del df["ВСП/КИЦ"]
del df["Блок"]

df.to_excel(f'pr_rev_KIC/Ревакцинация_КИЦ_{dd}.xlsx', index=False)

#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_KIC/Ревакцинация_КИЦ_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 8
ws.column_dimensions['C'].width = 16
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 38
ws.column_dimensions['F'].width = 35
ws.column_dimensions['G'].width = 20
rows = ws.max_row 
for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=2).value=='КИЦ':
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill
wb.save(f'pr_rev_KIC/Ревакцинация_КИЦ_{dd}.xlsx')
print('Для КИЦ готов')

#Управление безопасностью
df=pd.read_excel(f'pr_rev_BlokATB/Общий просрок_{dd}.xlsx')

df["ФИО"]=np.where(((df["Подразделение кратко"]=="Управление безопасности")|(df["Подразделение кратко"]=="Упр-ние безопасности")|(df["Подразделение кратко"]=="Отдел безопасности")), df["ФИО"], np.nan)
df=df[df["ФИО"].notnull()]

df=df.sort_values(by=["Подразделение кратко"])
df["Номер"]=range(1, len(df)+1)    
del df["ВСП/КИЦ"]
#del df["Блок 2"]

df.to_excel(f'pr_rev_BlokATB/Ревакцинация_Сервисы_УБ_{dd}.xlsx', index=False)
#изменение стиля текста и ширины столбца
shabl = str(Path.cwd() / f'pr_rev_BlokATB/Ревакцинация_Сервисы_УБ_{dd}.xlsx')
wb = openpyxl.load_workbook(shabl)
ws = wb.active
ws.column_dimensions['A'].width = 6
ws.column_dimensions['B'].width = 8
ws.column_dimensions['C'].width = 8
ws.column_dimensions['D'].width = 30
ws.column_dimensions['E'].width = 30
ws.column_dimensions['F'].width = 35
ws.column_dimensions['G'].width = 20
rows = ws.max_row 
for i in range(1, rows+1):
    for j in range(1, 8):
        ws.cell(row=i, column=j).font=Font(size=10, name ='Calibri')
        ws.cell(row=i, column=j).alignment=Alignment(horizontal='center', vertical='center', wrap_text=True)

rows = ws.max_row 
column=ws["G"]
fill=PatternFill(start_color="FFC0CB", end_color="FFC0CB", fill_type="solid")
for i in range(2, rows+1):
    if ws.cell(row=i, column=7).value!='' and ws.cell(row=i, column=3).value=='Сервисы':
        if ws.cell(row=i, column=7).value<0:
            ws.cell(row=i, column=7).fill=fill
wb.save(f'pr_rev_BlokATB/Ревакцинация_Сервисы_УБ_{dd}.xlsx')
print('Для УБ Сервисы готов')


Создание отчета общего по просрочке+прогноз на неделю
Для АТБ общий готов
Для АТБ ПРПА готов
Для АТБ РБиСП готов
Для АТБ КИБ готов
Для АТБ Сервисы готов
Для АТБ Блок Председателя готов
Для АТБ готов
Для ВО готов
Для КО готов
Для МЭ готов
Для НО готов
Для ПК готов
Для ПЦП НН готов
Для РМ готов
Для РТ готов
Для РУ готов
Для РЧ готов
Для КИЦ готов
Для УБ Сервисы готов
